In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Build Elo

In [36]:
from simulation_utils import db_get_data_for_latest_season

# Get data for the latest season
df = db_get_data_for_latest_season()

df

,season,home,away,utc_date,status,matchday,home_score,away_score,home_outcome,away_outcome,home_manager,away_manager,home_manager_count,away_manager_count,home_position,away_position
0,2023,Burnley FC,Manchester City FC,2023-08-11 19:00:00,FINISHED,1,0.0,3.0,0.0,3.0,Vincent Kompany,Pep Guardiola,0,0,2,1
1,2023,Arsenal FC,Nottingham Forest FC,2023-08-12 12:00:00,FINISHED,1,2.0,1.0,3.0,0.0,Mikel Arteta,Steve Cooper,0,0,1,4
2,2023,Sheffield United FC,Crystal Palace FC,2023-08-12 14:00:00,FINISHED,1,0.0,1.0,0.0,3.0,Paul Heckingbottom,Roy Hodgson,0,0,12,3
3,2023,Everton FC,Fulham FC,2023-08-12 14:00:00,FINISHED,1,0.0,1.0,0.0,3.0,Sean Dyche,Marco Silva,0,0,9,4
4,2023,Brighton & Hove Albion FC,Luton Town FC,2023-08-12 14:00:00,FINISHED,1,4.0,1.0,3.0,0.0,Roberto De Zerbi,Rob Edwards,0,0,2,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2023,Burnley FC,Nottingham Forest FC,2024-05-19 15:00:00,FINISHED,38,1.0,2.0,0.0,3.0,Vincent Kompany,Nuno Espírito Santo,0,1,19,17
376,2023,Brighton & Hove Albion FC,Manchester United FC,2024-05-19 15:00:00,FINISHED,38,0.0,2.0,0.0,3.0,Roberto De Zerbi,Erik ten Hag,0,0,13,8
377,2023,Crystal Palace FC,Aston Villa FC,2024-05-19 15:00:00,FINISHED,38,5.0,0.0,3.0,0.0,Oliver Glasner,Unai Emery,1,0,10,4
378,2023,Arsenal FC,Everton FC,2024-05-19 15:00:00,FINISHED,38,2.0,1.0,3.0,0.0,Mikel Arteta,Sean Dyche,0,0,2,12


## Test Against 2023-2024 Season

### Predict 2023-2024 Season

In [37]:
from simulation_utils import build_elo_before_season

# Build the elo dataframe before the current season
elo_df = build_elo_before_season(df)
elo_df

,elo,club_value,normalized_club_value,exponential_club_value,normalized_exponential_club_value,adjusted_elo
team,,,,,,
Manchester City FC,1650.602466,1.408200e+09,1.000000,1.718282,1.000000,1950.602466
Arsenal FC,1666.716148,1.172000e+09,0.821176,1.273171,0.740956,1889.002901
Chelsea FC,1527.029935,1.090200e+09,0.759246,1.136664,0.661512,1725.483603
Manchester United FC,1595.586633,8.181500e+08,0.553280,0.738948,0.430050,1724.601715
Liverpool FC,1560.801186,9.493000e+08,0.652572,0.920474,0.535695,1721.509574
Tottenham Hotspur FC,1575.404469,7.937000e+08,0.534769,0.707054,0.411489,1698.851216
Newcastle United FC,1587.402440,6.566500e+08,0.431010,0.538811,0.313576,1681.475155
Brighton & Hove Albion FC,1548.026512,5.782000e+08,0.371617,0.450077,0.261934,1626.606837
Brentford FC,1543.471265,4.197250e+08,0.251637,0.286129,0.166521,1593.427450


In [38]:
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path
import joblib
from tqdm import tqdm
from functools import partial

from simulation_utils import download_best_params_from_s3, download_model_and_scaler_from_s3, get_elo_dict_from_df, simulate_and_get_results

# Get the model and scalar from S3
model_file = Path("random_forest.joblib")
scaler_file = Path("standard_scaler.joblib")
download_model_and_scaler_from_s3(model_file, scaler_file)
model = joblib.load(model_file)
scaler = joblib.load(scaler_file)

# Get adjusted Elo dict
adjusted_elo = get_elo_dict_from_df(elo_df)

# Number of simulations to run
num_simulations = 10
# num_simulations = 1000

# Initialize a list to store results
seasons = []

# Get the best parameters for the model
best_params = download_best_params_from_s3()

# Create a partial function to pass the same arguments to each simulation
simulate_and_get_results_partial = partial(simulate_and_get_results, df=df, elo=adjusted_elo, model=model, scaler=scaler, k=best_params.k, half_life=best_params.half_life, decay_method=best_params.decay_method)

# Initialize a pool of workers
with ProcessPoolExecutor() as executor:
    seasons = list(tqdm(executor.map(simulate_and_get_results_partial, range(num_simulations)),  total=num_simulations, desc='Simulating', unit='season'))

# Unlink the model and scaler files
model_file.unlink()
scaler_file.unlink()

Model downloaded to random_forest.joblib
Scaler downloaded to standard_scaler.joblib


Simulating: 100%|██████████| 10/10 [00:02<00:00,  3.74season/s]


### Analyze Results Compared to actual 2023-2024 Season

### Post Process Simulation Results

In [40]:
import pandas as pd

# Aggregate the results
results = pd.concat(seasons).groupby("team").agg({"total_outcome": "sum"})

# Sort results based on total outcome
results = results.sort_values("total_outcome", ascending=False)

# Get the place each team finished in the league
results["place"] = range(1, len(results) + 1)

# For each season, get the place each team finished in the league
for index, season_df in enumerate(seasons):
    season_df["place"] = range(1, len(season_df) + 1)

# Get the average place each team finished in the league
average_results = pd.concat(seasons).groupby("team").agg({"place": "mean"}).sort_values("place")

# Get the season from seasons
list_of_seasons = df["season"].unique()

assert len(list_of_seasons) == 1, "All seasons should be the same"

# Get the season
simulation_season = list_of_seasons[0]

# Get a mapping of team names to a list places they finished in the league
team_place_mapping = {}
for team in average_results.index:
    team_place_mapping[team] = [season_df.loc[team, "place"] for season_df in seasons]

# Get the total number of seasons simulated
total_seasons = len(seasons)

# Get a mapping of times each team won the league
team_win_mapping = {}
for team in average_results.index:
    team_win_mapping[team] = sum([season_df.loc[team, "place"] == 1 for season_df in seasons]) / total_seasons

# Get a mapping of times each team finished in the top 4
team_top_4_mapping = {}
for team in average_results.index:
    team_top_4_mapping[team] = sum([season_df.loc[team, "place"] <= 4 for season_df in seasons]) / total_seasons

# Get a mapping of times each team finished in the bottom 3
team_bottom_3_mapping = {}
for team in average_results.index:
    team_bottom_3_mapping[team] = sum([season_df.loc[team, "place"] > len(season_df) - 3 for season_df in seasons]) / total_seasons

# Build a dataframe with the average place, times won, times in top 4, and times in bottom 3
average_results["win_premier_league"] = [team_win_mapping[team] for team in average_results.index]
average_results["top_4"] = [team_top_4_mapping[team] for team in average_results.index]
average_results["bottom_3"] = [team_bottom_3_mapping[team] for team in average_results.index]

# Show the dataframe
average_results

,place,win_premier_league,top_4,bottom_3
team,,,,
Manchester City FC,1.0,1.0,1.0,0.0
Arsenal FC,2.0,0.0,1.0,0.0
Liverpool FC,3.0,0.0,1.0,0.0
Aston Villa FC,4.0,0.0,1.0,0.0
Tottenham Hotspur FC,5.0,0.0,0.0,0.0
Chelsea FC,6.0,0.0,0.0,0.0
Manchester United FC,7.0,0.0,0.0,0.0
Newcastle United FC,8.0,0.0,0.0,0.0
West Ham United FC,9.0,0.0,0.0,0.0


### Find the Positions of the Teams in the 2023-2024 Season for all Seasons

In [41]:
team_to_position = {team: {} for team in df["home"].unique()}
for season in seasons:
    # Assign the position of each team
    for i, team in enumerate(season.index, 1):
        if i not in team_to_position[team]:
            team_to_position[team][i] = 0
        team_to_position[team][i] += 1

# Create a dataframe where each row is a team and the one column is an array of positions in which the team finished in the league
team_positions_df = pd.DataFrame.from_dict(team_to_position, orient="index").fillna(0).astype(int)

# Stack the dataframe to get a row for each team-position pair
team_positions_df = team_positions_df.stack().reset_index().rename(columns={'level_0': 'team', 'level_1': 'position', 0: 'count'})

# Set the index to the team name-position pair
team_positions_df.set_index(["team", "position"], inplace=True)

# Sort the position index
team_positions_df = team_positions_df.sort_index()

team_positions_df

count
team                       position       
AFC Bournemouth            1             0
                           2             0
                           3             0
                           4             0
                           5             0
...                                    ...
Wolverhampton Wanderers FC 16            0
                           17            0
                           18            0
                           19            0
                           20            0

[400 rows x 1 columns]

## Store the Data

In [42]:
from simulation_utils import db_store_results

# Store the results in the database
db_store_results(simulation_season, average_results, team_positions_df)